In [ ]:
# default_exp core

# docments.core

> API details.

In [ ]:
#export
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef
from io import BytesIO
from textwrap import dedent
from fastcore.basics import *
import inspect,re

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def _parses(s):
    "Parse Python code in string or function object `s`"
    if inspect.isfunction(s): s = inspect.getsource(s)
    return parse(s)

In [ ]:
#export
def param_locs(s):
    "`dict` of parameter line numbers to names"
    body = _parses(s).body
    assert len(body)==1,"Body must contain exactly one definition"
    assert isinstance(body[0], FunctionDef),"Body must contain a function definition"
    defn = body[0]
    return {defn.returns.lineno:'return', **{arg.lineno:arg.arg for arg in defn.args.args}}

The location of the closing parenthesis for the parameter list is in the `return` key.

In [ ]:
def adder(
    a:int, # The 1st number to add
    # The 2nd number to add.
    #   NB: This is added to `a`
    b:int,
)->int:    # The result of adding `a` to `b`
    "Add `a` to `b`"
    # Calculate the addition
    return a+b

parms = param_locs(adder)
parms

{6: 'return', 2: 'a', 5: 'b'}

In [ ]:
#export
def _tokens(s):
    "Tokenize Python code in string or function object `s`"
    if inspect.isfunction(s): s = inspect.getsource(s)
    return tokenize(BytesIO(s.encode('utf-8')).readline)

_clean_re = re.compile('^\s*#(.*)\s*$')
def _clean_comment(s):
    res = _clean_re.findall(s)
    return res[0] if res else None

def _get_comment(line, arg, comments, parms):
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

In [ ]:
#export
def docments(s):
    "`dict` of parameter names to 'docment-style' comments in function or string `s`"
    comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
    parms = param_locs(s)
    return {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}

In [ ]:
docments(adder)

{'return': 'The result of adding `a` to `b`',
 'a': 'The 1st number to add',
 'b': 'The 2nd number to add.\n  NB: This is added to `a`'}

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
